<a href="https://colab.research.google.com/github/goerlitz/nlp-classification/blob/main/notebooks/10kGNAD/colab/21d_10kGNAD_optuna_trial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of Optuna Results

In [65]:
!pip install -q --upgrade optuna >/dev/null

In [66]:
import optuna
from optuna.storages import RDBStorage

## Connect Google Drive

In [67]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Connect Database

https://optuna.readthedocs.io/en/stable/reference/generated/optuna.storages.RDBStorage.html#optuna.storages.RDBStorage

In [68]:
db_path = "/content/gdrive/My Drive/Colab Notebooks/nlp-classification/"
db_name = "10kgnad_hf__distilbert-base-german-cased"

storage = RDBStorage(
    url=f"sqlite:///{db_path}{db_name}.db",
    #engine_kwargs={"pool_size": 20, "connect_args": {"timeout": 10}},
)

In [69]:
for s in storage.get_all_study_summaries():
    print(s.datetime_start, s.study_name, s.n_trials, s.best_trial)

2021-12-17 08:16:53.667223 10kgnad_hf__distilbert-base-german-cased 33 FrozenTrial(number=28, values=[0.3278708755970001], datetime_start=datetime.datetime(2021, 12, 17, 21, 57, 46, 766614), datetime_complete=datetime.datetime(2021, 12, 17, 22, 8, 24, 714547), params={'learning_rate': 6.496217126578442e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'weight_decay': 0.0031495603091898777}, distributions={'learning_rate': LogUniformDistribution(high=0.0001, low=1e-05), 'num_train_epochs': CategoricalDistribution(choices=(2,)), 'per_device_train_batch_size': CategoricalDistribution(choices=(8, 16, 32)), 'weight_decay': LogUniformDistribution(high=0.01, low=0.001)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=29, state=TrialState.COMPLETE, value=None)


In [70]:
study = storage.get_all_study_summaries()[0].study_name
trials = storage.get_all_trials(storage.get_study_id_from_name(study))

In [71]:
trials[1]

FrozenTrial(number=1, values=[0.37806227803230286], datetime_start=datetime.datetime(2021, 12, 17, 8, 17, 25, 11382), datetime_complete=datetime.datetime(2021, 12, 17, 8, 28, 56, 206016), params={'learning_rate': 3.725801318786445e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 16, 'weight_decay': 0.006507560517674187}, distributions={'learning_rate': LogUniformDistribution(high=0.0001, low=1e-05), 'num_train_epochs': CategoricalDistribution(choices=(2,)), 'per_device_train_batch_size': CategoricalDistribution(choices=(8, 16, 32)), 'weight_decay': LogUniformDistribution(high=0.01, low=0.001)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)

In [72]:
from optuna.trial import TrialState

complete_trials = [t for t in trials if t.state==TrialState.COMPLETE]

trial_df = pd.DataFrame([t.params for t in complete_trials])
trial_df.index = [t.number for t in complete_trials]
trial_df["eval_loss"] = [t.value for t in complete_trials]
trial_df.sort_values("eval_loss")

,learning_rate,num_train_epochs,per_device_train_batch_size,weight_decay,eval_loss
28,0.000065,2,32,0.003150,0.327871
22,0.000062,2,32,0.006756,0.329290
24,0.000060,2,32,0.005864,0.329652
26,0.000064,2,32,0.004508,0.332105
23,0.000068,2,32,0.007089,0.333058
16,0.000089,2,32,0.009658,0.338841
20,0.000074,2,32,0.007228,0.339404
15,0.000084,2,32,0.009690,0.339937
25,0.000053,2,32,0.005767,0.341159
19,0.000076,2,32,0.008290,0.343946


In [73]:
import plotly.express as px
df = px.data.iris()
fig = px.parallel_coordinates(trial_df, color="eval_loss", 
                             color_continuous_scale=px.colors.sequential.Oryel)
fig.show()